<a href="https://colab.research.google.com/github/nayakiniki/ML-tasks/blob/main/Restaurantrecommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
# Step 1: Importing necessary libraries
import pandas as pd
import io
from google.colab import files
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [26]:
# Step 2: Uploading given dataset
uploaded = files.upload()
filename = list(uploaded.keys())[0]

Saving Dataset .csv to Dataset  (1).csv


In [27]:
# Step 3: Read the CSV file
df = pd.read_csv(io.BytesIO(uploaded[filename]), encoding='latin-1')
df.head()

,ï»¿Restaurant ID,Restaurant Name,Country Code,City,Address,Locality,Locality Verbose,Longitude,Latitude,Cuisines,...,Currency,Has Table booking,Has Online delivery,Is delivering now,Switch to order menu,Price range,Aggregate rating,Rating color,Rating text,Votes
0,6317637,Le Petit Souffle,162,Makati City,"Third Floor, Century City Mall, Kalayaan Avenu...","Century City Mall, Poblacion, Makati City","Century City Mall, Poblacion, Makati City, Mak...",121.027535,14.565443,"French, Japanese, Desserts",...,Botswana Pula(P),Yes,No,No,No,3,4.8,Dark Green,Excellent,314
1,6304287,Izakaya Kikufuji,162,Makati City,"Little Tokyo, 2277 Chino Roces Avenue, Legaspi...","Little Tokyo, Legaspi Village, Makati City","Little Tokyo, Legaspi Village, Makati City, Ma...",121.014101,14.553708,Japanese,...,Botswana Pula(P),Yes,No,No,No,3,4.5,Dark Green,Excellent,591
2,6300002,Heat - Edsa Shangri-La,162,Mandaluyong City,"Edsa Shangri-La, 1 Garden Way, Ortigas, Mandal...","Edsa Shangri-La, Ortigas, Mandaluyong City","Edsa Shangri-La, Ortigas, Mandaluyong City, Ma...",121.056831,14.581404,"Seafood, Asian, Filipino, Indian",...,Botswana Pula(P),Yes,No,No,No,4,4.4,Green,Very Good,270
3,6318506,Ooma,162,Mandaluyong City,"Third Floor, Mega Fashion Hall, SM Megamall, O...","SM Megamall, Ortigas, Mandaluyong City","SM Megamall, Ortigas, Mandaluyong City, Mandal...",121.056475,14.585318,"Japanese, Sushi",...,Botswana Pula(P),No,No,No,No,4,4.9,Dark Green,Excellent,365
4,6314302,Sambo Kojin,162,Mandaluyong City,"Third Floor, Mega Atrium, SM Megamall, Ortigas...","SM Megamall, Ortigas, Mandaluyong City","SM Megamall, Ortigas, Mandaluyong City, Mandal...",121.057508,14.584450,"Japanese, Korean",...,Botswana Pula(P),Yes,No,No,No,4,4.8,Dark Green,Excellent,229


In [28]:
# Step 4: Drop rows with missing values in key columns
df = df.dropna(subset=['Restaurant Name', 'Cuisines', 'City', 'Average Cost for two', 'Aggregate rating'])

In [29]:
# Step 5: Normalize and combine features for similarity
df['Cuisines'] = df['Cuisines'].astype(str).str.lower().str.strip()
df['City'] = df['City'].astype(str).str.lower().str.strip()
df['Average Cost for two'] = df['Average Cost for two'].astype(str).str.strip()

df['content'] = df['Cuisines'] + ' ' + df['City'] + ' ' + df['Average Cost for two']


In [30]:
# Step 6: Vectorize the content
vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = vectorizer.fit_transform(df['content'])

In [31]:
# Step 7: Compute cosine similarity
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)


In [32]:
# Step 8: Define recommendation function
def get_recommendations(restaurant_name, cosine_sim=cosine_sim, top_n=10):
    restaurant_name = restaurant_name.strip().lower()
    df['Restaurant Name'] = df['Restaurant Name'].astype(str).str.strip().str.lower()

    if restaurant_name not in df['Restaurant Name'].values:
        print(f"\n❌ Restaurant '{restaurant_name}' not found in the dataset.")
        return None

    idx = df[df['Restaurant Name'] == restaurant_name].index[0]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[1:top_n + 1]
    restaurant_indices = [i[0] for i in sim_scores]

    return df[['Restaurant Name', 'Cuisines', 'City', 'Average Cost for two', 'Aggregate rating']].iloc[restaurant_indices]

In [37]:
# Step 9: Example usage
restaurant_name = "Izakaya Kikufuji"
recommendations = get_recommendations(restaurant_name)


In [38]:

# Step 10: Display recommendations
if recommendations is not None:
    print("\n✅ Top Similar Restaurants:\n")
    print(recommendations)


✅ Top Similar Restaurants:

                   Restaurant Name                    Cuisines  \
0                 le petit souffle  french, japanese, desserts   
1537                       komachi                    japanese   
3184                      aka saka           chinese, japanese   
3238                 moets oh! bao     chinese, japanese, thai   
6139                       ichiban     chinese, thai, japanese   
1169                           red     japanese, thai, chinese   
8102       aim cafe and restaurant             japanese, sushi   
480   fuji bay japanese restaurant             japanese, sushi   
9334                      wagamama             japanese, asian   
3100                       fa yian                     chinese   

                 City Average Cost for two  Aggregate rating  
0         makati city                 1100               4.8  
1537          gurgaon                 1200               3.2  
3184        new delhi                 1200             